In [2]:
#Bibliotecas
import numpy as np
import cv2
import imutils
import pytesseract

verbose = True

# Imagem original
image = cv2.imread('1.jpg')

#Pré-Processamento
# Resize a imagem - mudar width para 500
image = imutils.resize(image, width=500)

# Show Imagem Original
if verbose == True:
    cv2.imshow("Imagem Original ", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# RGB -> Tons de cinza
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

if verbose == True:
    cv2.imshow("1 - Conversao Tons de cinza", gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


#Processamento de Imagem
# Remoção de ruído com filtro bilateral iterativo (remove o ruído enquanto preserva as bordas)
gray = cv2.bilateralFilter(gray, 11, 17, 17)

if verbose == True:
    cv2.imshow("2 - Filtro Bilateral ", gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Encontrar bordas da Imagem em tons de cinza
edged = cv2.Canny(gray, 170, 200)

if verbose == True:
    cv2.imshow("3 - Canny Edges", edged)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Encontrar contornos baseado nas bordas
cnts, _  = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Crie uma cópia da imagem original para desenhar todos os contornos
img1 = image.copy()
cv2.drawContours(img1, cnts, -1, (0,255,0), 3)

if verbose == True:
    cv2.imshow("4 - All Contours", img1)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


#classifique os contornos com base em sua área, mantendo a área mínima exigida como '30' (qualquer coisa menor que isso não será considerada)
cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30]
NumPlacaCnt = None #Sem contorno da placa

# Top 30 Contornos
img2 = image.copy()
cv2.drawContours(img2, cnts, -1, (0,255,0), 3)

if verbose == True:
    cv2.imshow("5 - Top 30 Contours", img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# loop sobre os contornos para encontrar a melhor aproximacao do contorno do numero da placa
count = 0
idx = 1
for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # print ("approx = ",approx)
        if len(approx) == 4:  # Selecionar contorno com 4 bordas
            NumPlacaCnt = approx #Aproximacao da placa

            # Cortar esses contornos e armazená-lo na pasta Imagens recortadas
            x, y, w, h = cv2.boundingRect(c) #Encontrar as coorde da placa
            new_img = gray[y:y + h, x:x + w] #Criar uma nova imagem
            cv2.imwrite('Placa' + str(idx) + '.png', new_img) #Armazenar a imagem
            break


# Desenhando o contorno selecionado na imagem original
#print(NumberPlateCnt)
cv2.drawContours(image, [NumPlacaCnt], -1, (0,255,0), 3)

if verbose == True:
    cv2.imshow("Imagem final com a placa detectada", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Cropped_img_loc = 'Placa' + str(idx) + '.png'

if verbose == True:
    cv2.imshow("Cropped Image ", cv2.imread(Cropped_img_loc))
    cv2.waitKey(0)
    cv2.destroyAllWindows()




### Tesseract comandos básico

**pytesseract.image_to_string(InputImage, config='OCR-Language, OCR Engine Mode (oem), Page Segmentation Mode (psm)')**

1. **InputImage** = Imagem que queremos trabalhar.
2. **OCR-Language** = Linguagem básica é o Ingles. Se quiser usar outra linguagem é preciso fazer o download do outro idioma. Depois do download é preciso acessar a pasta **'tessdata'**, provavlemnte em **'C:\Arquivos de Programas\tesseract-OCR\tessdata'** e adicionar o arquivo traineddata.
3. **OCR Engine Mode (oem)** = Tesseract possui 4 modos de operação da OCR engine.
    * 0    Legacy engine only.
    * 1    Neural nets LSTM engine only.
    * 2    Legacy + LSTM engines.
    * 3    Default, based on what is available.
4. **Page Segentation Mode (psm)** = O PSM pode ser muito útil quando você tiver informações adicionais sobre a estrutura do texto. Existe 11 modos.
      * 0 = Orientation and script detection (OSD) only.
      * 1 = Automatic page segmentation with OSD.
      * 2 = Automatic page segmentation, but no OSD, or OCR.
      * 3 = Fully automatic page segmentation, but no OSD. (Default)
      * 4 = Assume a single column of text of variable sizes.
      * 5 = Assume a single uniform block of vertically aligned text.
      * 6 = Assume a single uniform block of text.
      * 7 = Treat the image as a single text line.
      * 8 = Treat the image as a single word.
      * 9 = Treat the image as a single word in a circle.
      * 10 = Treat the image as a single character.

In [7]:
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files (x86)\\Tesseract-OCR\\tesseract.exe' #Linha pra ser usada em amb Windows

text = pytesseract.image_to_string(Cropped_img_loc)

print("Numero é :", text)

Numero é : L BRA 1234 1
